# Using an output handler for validating agent's output

As no one can guarantee the validity or integrity of a raw LLM output, getting a sensible and/or structured output from an agent is tricky. In motleycrew, we addressed this issue with a novel approach: please welcome **output handlers**.

As of now, tool calling is the most reliable way to get a structured output from an LLM. So why not have an output handler - a regular tool with an input schema and a description which instructs the agent to return the output only via this tool. Moreover, we can force the agent into doing this: if it attempts to finish the regular way, we intercept the message and remind the agent that it must call the tool instead.

In [1]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import StructuredTool

from motleycrew import MotleyCrew
from motleycrew.agents.langchain import ReActToolCallingMotleyAgent
from motleycrew.common import configure_logging
from motleycrew.tasks import SimpleTask

from motleycrew.common.exceptions import InvalidOutput

from motleycrew.tools import MotleyTool

We'll be prompting our agent to create a report on the latest advancements in AI.  
Say we want it to also include some info on AI applications in physics if it did not already. So our output handler will look for the keyword "physics" in the agent's output.

Also, we can do any data manipulations we want inside the output handler. In this case, let's just place the output into a certain field.

In [2]:
def check_output(output: str):
    if "physics" not in output.lower():
        raise InvalidOutput("Add more information about AI applications in physics.")

    return {"checked_output": output}

output_handler = StructuredTool.from_function(
    name="output_handler",
    description="Output handler",
    func=check_output,
)  # Schema is inferred from the function signature, but you can also specify it explicitly

# To set return_direct=True, we'll convert it into a MotleyTool
output_handler = MotleyTool.from_supported_tool(output_handler, return_direct=True)

Now let's create our agent and launch the crew!

In [3]:
search_tool = DuckDuckGoSearchRun()
researcher = ReActToolCallingMotleyAgent(
    tools=[search_tool, output_handler],
    verbose=True,
    chat_history=True,
    force_output_handler=True,  # This will force the agent to use the output handler instead of regular output
)

crew = MotleyCrew()
task = SimpleTask(
    crew=crew,
    name="produce comprehensive analysis report on AI advancements in 2024",
    description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
Identify key trends, breakthrough technologies, and potential industry impacts.
Perform no more than 3 search queries. Your final answer MUST be a full analysis report""",
    agent=researcher,
)

In [4]:
crew.run()



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'latest advancements in AI 2024'}`
responded: Thought: To conduct a comprehensive analysis of the latest advancements in AI in 2024, I need to gather information on key trends, breakthrough technologies, and potential industry impacts. I will start by performing a search query to get an overview of the latest advancements in AI in 2024. This will help identify the key areas to focus on for further detailed searches.

I will perform the first search query to get an overview of the latest advancements in AI in 2024.

Calling the tool to perform the search query.



In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinkering with a million little AI models. State-of-the-art AI models ... The Top 5 Artificial Intelligence (AI) Trends For 2024. Adobe Stock. It's been a year since OpenAI released ChatGPT, opening the door to seamlessly


Invoking: `output_handler` with `{'output': '### Comprehensive Analysis of the Latest Advancements in AI in 2024\n\n#### Key Trends in AI 2024\n\n1. **Generative AI**: Generative AI continues to evolve, becoming more useful for everyday users and businesses. The technology is being integrated into various applications, from creative tools to business solutions, enabling more people to experiment with and benefit from AI models.\n\n2. **Multimodal AI**: This trend involves AI systems that can process and integrate multiple types of data inputs, such as text, images, and sound. This advancement mimics human sensory processing and enhances the capabilities of AI in understanding and generating complex data.\n\n3. **Smaller Language Models and Open Source Advancements**: There is a shift towards smaller, more efficient language models that are easier to deploy and manage. Open source contributions are accelerating innovation and making advanced AI technologies more accessible.\n\n4. **Mod

[TaskUnit(status=done)]

As you can see in the logs, our physics requirement was successfully fed back to the agent!

In [5]:
from IPython.display import display, Markdown

final_result = task.output["checked_output"]
display(Markdown(final_result))

### Comprehensive Analysis of the Latest Advancements in AI in 2024

#### Key Trends in AI 2024

1. **Generative AI**: Generative AI continues to evolve, becoming more useful for everyday users and businesses. The technology is being integrated into various applications, from creative tools to business solutions, enabling more people to experiment with and benefit from AI models.

2. **Multimodal AI**: This trend involves AI systems that can process and integrate multiple types of data inputs, such as text, images, and sound. This advancement mimics human sensory processing and enhances the capabilities of AI in understanding and generating complex data.

3. **Smaller Language Models and Open Source Advancements**: There is a shift towards smaller, more efficient language models that are easier to deploy and manage. Open source contributions are accelerating innovation and making advanced AI technologies more accessible.

4. **Model Optimization and Customization**: AI models are becoming more optimized and customizable, allowing businesses to tailor AI solutions to their specific needs. This trend is driven by advancements in model optimization techniques and the development of customized local models and data pipelines.

5. **Integration into Daily Life and Business**: AI is becoming seamlessly integrated into daily life and business operations. From AI-powered chatbots to advanced analytics, organizations are leveraging AI to drive efficiency, innovation, and value.

#### Breakthrough AI Technologies in 2024

1. **Generative AI Models**: State-of-the-art generative AI models are being developed and deployed, offering new capabilities in content creation, design, and problem-solving.

2. **AI for Precision Education**: AI is making significant strides in the field of education, with systems designed to provide personalized learning experiences and improve educational outcomes.

3. **AI in Climate Research**: AI technologies are being used to advance climate research, helping scientists analyze vast amounts of data and develop solutions to address climate change.

4. **AI for Social Good**: There is a growing focus on using AI for social good, with tech leaders and social innovators working together to develop AI solutions that address societal challenges.

5. **AI in Physics**: AI is being used to uncover mechanisms in physics, driving new paradigms in AI discovery. Researchers are using generative AI to classify phase transitions in materials and physical systems more efficiently. AI is also being coupled with fundamental physics to enhance calculations and theoretical investigations in pure mathematics and theoretical physics.

#### Potential Industry Impacts

1. **Business and Enterprise**: AI is transforming business operations, from automating routine tasks to providing advanced analytics and insights. Companies are using AI to enhance productivity, drive innovation, and gain a competitive edge.

2. **Healthcare**: AI is revolutionizing healthcare by improving diagnostics, personalizing treatment plans, and streamlining administrative processes. AI-powered tools are enhancing patient care and outcomes.

3. **Education**: AI is reshaping education by offering personalized learning experiences and improving access to quality education. AI-driven tools are helping educators tailor instruction to individual student needs.

4. **Climate and Environment**: AI is playing a crucial role in climate research and environmental conservation. AI technologies are helping researchers analyze data, model climate scenarios, and develop sustainable solutions.

5. **Social Impact**: AI is being leveraged to address social issues, from improving public safety to enhancing accessibility for individuals with disabilities. AI-driven initiatives are making a positive impact on communities worldwide.

6. **Physics**: AI applications in physics are leading to significant advancements in understanding and modeling physical phenomena. The integration of AI with fundamental physics is enabling faster and more efficient calculations, as well as new insights in theoretical investigations.

In conclusion, 2024 is a pivotal year for AI, with significant advancements and trends shaping the future of technology and its applications across various industries. The integration of AI into daily life and business, coupled with breakthrough technologies, is driving innovation and creating new opportunities for growth and development.